## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
! pip install pyspark

In [0]:
%run ./includes/includes

### VERSION 2 VARIABLES
I have created two versions of checkpoints and delta files for each stage because I encountered some issues when trying to insert my logs. It is possible that the data got mixed up during the process, so I decided to use version 2 variables to ensure accuracy and consistency. But the code will be executed with the original variables. <br>

< Optimization Variables>
- BRONZE_CHECKPOINT_2 & BRONZE_DELTA_2 (*please ignore*)
- SILVER_CHECKPOINT_2 & SILVER_DELTA_2 (*please ignore*)
- GOLD_CHECKPOINT_2 & GOLD_DELTA_2 (*please ignore*)



In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# Standard and enhanced imports for PySpark functionality
import time
import matplotlib.pyplot as plt
import mlflow
from mlflow.pyfunc import spark_udf
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# PySpark SQL functions and types
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType
from pyspark.sql.window import Window

# Combining multiple PySpark SQL functions into one import statement
from pyspark.sql.functions import (
    input_file_name, current_timestamp, isnan, isnull, when, count, desc, col, to_date, to_timestamp, explode,
    split, length, regexp_replace, expr, countDistinct, sum as spark_sum
)
from delta.tables import *
start_time = time.time()


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE

data_frame_files = get_source_listing_df()
print(data_frame_files)

file_count = len(data_frame_files) - 1
print(f"Number of source files: {file_count}")

if file_count > 0:
    name = data_frame_files.iloc[1]['File Name']
    print(f"\nContents of {name}:")
    contents = show_s3_file_contents(name)
    print(contents.decode('utf-8'))
else:
    print("FILES NOT EXISTED")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
#ENTER YOUR CODE HERE
inputDataSchema = StructType([
    StructField('date', StringType(), True),
    StructField('sentiment', StringType(), True),
    StructField('text', StringType(), True),
    StructField('user', StringType(), True),
])
dbutils.fs.rm(USER_DIR + "/bronze.delta", True)
dbutils.fs.rm(USER_DIR + "/bronze.checkpoint", True)


# READ DATA STREAM
raw_data_stream = (spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("maxFilesPerTrigger", 1)
        .schema(inputDataSchema) 
        .load(TWEET_SOURCE_PATH)
        .selectExpr("*","_metadata.file_name as source_file_name","_metadata.file_modification_time as processing_time_stamp"))    
activity_query = (raw_data_stream.writeStream
        .queryName("bronze_stream") 
        .format("delta")
        .option("checkpointLocation", BRONZE_CHECKPOINT)
        .option("mergeSchema", "true")
        .outputMode("append") 
        .start(BRONZE_DELTA) )
    
# activity_query.awaitTermination(100)
# activity_query.stop()


#Show 10 records
bronze_table = spark.read.load(BRONZE_DELTA)
print(bronze_table.show(10))

In [0]:
%sql
SET spark.sql.legacy.timeParserPolicy = LEGACY;

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# HOW MANY TWEETS IN BRONZE TABLE
total_tweets_bronze = bronze_table.count()
print("Total Number of rows = " , total_tweets_bronze)

# HOW MANY COLUMNS OF NAN VALUE IN BRONZE TABLE
dict = {col:bronze_table.filter(bronze_table[col].isNull()).count() for col in bronze_table.columns}
print("Count of columns having nan/null column-wise")
print(dict)

print("There is no Nan/ Null values in Bronze Table")
print("If there is nan/ Null values, I will remove them or assign the average values into them if the values are int. ")

# COUNT THE NUMBERS OF TWEETS
input_table = spark.read.format("delta").load(BRONZE_DELTA)
query = input_table.groupBy("user").count().orderBy("count", ascending=False)
print(query.show(10))

# MENTION COUNTS
mention_count_bronze = input_table.filter(col("text").like("%@%")).count()
zero_tweets_bronze = total_tweets_bronze - mention_count_bronze
print("Number of tweets with at least 1 mention: ", mention_count_bronze)

print("Number of tweets with 0 mentions: ", mention_count_bronze)


# PLOT TOP USERS
top_users = query.limit(20).toPandas()

plt.figure(figsize=(30, 10))
plt.bar(top_users['user'], top_users['count'])
plt.xlabel('Users Name')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Users')
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# CLEAN UP PREVIOUS DATA
dbutils.fs.rm(USER_DIR + "/silver.delta", True)
dbutils.fs.rm(USER_DIR + "/silver.checkpoint", True)

In [0]:
# READ BRONZE TABLE
read_bronze_stream =  (
        spark.readStream
        .format("delta")
        .load(BRONZE_DELTA)
    )


# READ BRONZE STREAM
read_bronze_stream = ( read_bronze_stream
                      .withColumn("timestamp", to_date(read_bronze_stream["date"], "EEE MMM dd HH:mm:ss zzz yyyy"))
                      .withColumn("mentions", explode(split(col("text"), " "))).filter(col("mentions").startswith("@")).filter(col("mentions") != "@")
                      .withColumn("cleaned_text", regexp_replace(col("text"), "@[a-zA-Z0-9_]+", "")).select("timestamp", "mentions", "cleaned_text", "sentiment"))


# WRITE STREAM TO SILVER 
write_silver_stream = (
            read_bronze_stream.writeStream.format("delta")
                    .queryName("silver_stream") 
                    .option("checkpointLocation", SILVER_CHECKPOINT)
                    .outputMode("append")
                    .start(SILVER_DELTA)
             )


# write_silver_stream.awaitTermination(80)
# write_silver_stream.stop()

silver_table = spark.read.load(SILVER_DELTA)
print(silver_table.show(20))

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# CHECKING DIRECTORY 
dbutils.fs.ls(USER_DIR)


In [0]:
# REMOVE PREVIOUS DATA

dbutils.fs.rm(USER_DIR + "/gold.delta", True)
dbutils.fs.rm(USER_DIR + "/gold.checkpoint", True)

In [0]:
# ENTER YOUR CODE HERE
# SENTIMENT MODEL
sent_model = spark_udf(spark, "models:/" + MODEL_NAME + "/production")


# READ STREAM GOLD
read_silver_stream = (
        spark.readStream
        .format("delta")
        .load(SILVER_DELTA)
)
# COMMENT FOR SENTIMENT ANALYSIS : For neutral comments, I will handle them as negative as I've found that it will not be any big influences on the analysis.

read_silver_stream = (read_silver_stream.withColumn("predicted", sent_model("cleaned_text"))
                      .select("timestamp", "mentions", "cleaned_text", "sentiment", col("predicted.score").alias("predicted_score"), col("predicted.label").alias("predicted_sentiment"))
                      .withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(-1))
                      .withColumn("predicted_sentiment_id", when(col("predicted_sentiment") == "POS", 1).otherwise(-1)))

# WRITE STREAM GOLD
write_gold_stream = (read_silver_stream.writeStream.format("delta")
                    .queryName("gold_stream")
                    .option("checkpointLocation", GOLD_CHECKPOINT)
                    .outputMode("append")
                    .start(GOLD_DELTA))


# write_gold_stream.awaitTermination(200)
# write_gold_stream.stop()

In [0]:
gold_table = spark.read.load(GOLD_DELTA)
print(gold_table.show(20))

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE

# PREDICTED RESULT WITH SENTIMENT MODEL
predicted_result = gold_table.withColumn("predicted_sentiment_id", when(col("predicted_sentiment_id") == 0, -1).otherwise(col("predicted_sentiment_id")))
predicted_sentiment = predicted_result.select("predicted_sentiment_id").collect()

# CONFUSION MATRIX
predicted_sentiment_ids = gold_table.select("predicted_sentiment_id").collect()
true_sentiment_ids = gold_table.select("sentiment_id").collect()

recall = recall_score(true_sentiment_ids, predicted_sentiment_ids)
f1 = f1_score(true_sentiment_ids, predicted_sentiment_ids)
precision = precision_score(true_sentiment_ids, predicted_sentiment_ids)

print("recall =", recall)
print("f1 =", f1)
print("model_precesion =", precision)

confusion_mtx = confusion_matrix(true_sentiment_ids, predicted_sentiment_ids)
my_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx, display_labels=["Negative", "Positive"])
my_confusion_matrix.plot()

# SAVE TO THE FOLDER
dbutils.fs.mkdirs("dbfs:/images")
my_confusion_matrix.figure_.savefig("/dbfs/images/confusion_matrix.PNG")

# ML FLOW
silver_table_version = DeltaTable.forPath(spark, SILVER_DELTA).history().toPandas()["version"].iloc[-1]

with mlflow.start_run():
    mlflow.log_param("MODEL_NAME", MODEL_NAME)
    mlflow.log_param("mlflow_version", mlflow.version.VERSION)
    mlflow.log_param("silver_table_version", silver_table_version)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("f1", f1)


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE
distinct_mentions = gold_table.select(countDistinct("mentions")).collect()[0][0]
print("Total Mentions is:", distinct_mentions)

grouped_mentions_df = gold_table.groupBy("mentions", "sentiment").count()
pivot_df = grouped_mentions_df.groupBy("mentions").pivot("sentiment").sum("count")
count_mentions = pivot_df.na.fill(0)
count_mentions.show()

window_spec = Window.partitionBy("mentions")
count_mentions = count_mentions.withColumn("total", spark_sum(count_mentions["positive"] + count_mentions["negative"]).over(window_spec))
count_mentions.show()

sorted_count_mentions = count_mentions.orderBy(count_mentions["total"].desc())
sorted_count_mentions.show()


mentions_top_pos = sorted_count_mentions.filter(sorted_count_mentions["positive"] > 0).limit(20)
mentions = mentions_top_pos.select("mentions").rdd.flatMap(lambda x: x).collect()
positive_counts = mentions_top_pos.select("positive").rdd.flatMap(lambda x: x).collect()
plt.figure(figsize=(20, 10))
plt.bar(mentions, positive_counts)
plt.xlabel("Mentions_POS")
plt.ylabel("Count")
plt.title("Top 20 Mentions POS_Sentiment")
plt.xticks(rotation=90)
plt.show()

mentions_top_neg = sorted_count_mentions.filter(sorted_count_mentions["negative"] > 0).limit(20)
mentions = mentions_top_neg.select("mentions").rdd.flatMap(lambda x: x).collect()
negative_counts = mentions_top_neg.select("negative").rdd.flatMap(lambda x: x).collect()
plt.figure(figsize=(20, 10))
plt.bar(mentions, negative_counts)
plt.xlabel("Mentions_NEG")
plt.ylabel("Count")
plt.title("Top 20 Mentions NEG_Sentiment")
plt.xticks(rotation=90)
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

# CHECK ACTIVE STREAMS
active_streams = spark.streams.active
for stream in active_streams:
    print(stream.name)

# STOP ALL ACTIVE STREAMS
for stream in spark.streams.active:
    print(f"Stopping stream: {stream.name}")
    stream.stop()

# ELAPSED TIME CHECK
elapsed_time = time.time() - start_time
print("Elapsed time:", elapsed_time)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE

## Optimization Logs of Spark Application

The process of creating the Gold Delta table has been consistently marred by errors. Initially, I attributed this to a need for more time to establish a stream. However, upon delving into the log, I uncovered a more intricate issue-the potential need for executor adjustment.

My investigation was comprehensive, focusing on tasks that spill data to disk. This is a common occurrence when memory capacity is insufficient for operations like joins or aggregations. Despite the absence of streaming initiation, I meticulously examined my executor logs. 

<br><br>
<img src="/files/tables/DSCC402/failed_tasks-1.png" alt="error_no_space" width="75%"><br><br>
Checking out executors' failure as below... <br>
<img src="/files/tables/DSCC402/executor_failure.png" alt="error_no_space" width="75%">

<br><br>

If I check the executors log, I can confirm that the task has not been completed. <br><br>
<img src="/files/tables/DSCC402/executor_stages_failure-1.png" alt="error_no_space" width="50%">

<br>
<br>
Also, I could confirm that the errors as following: 


![error_no_space](/files/tables/DSCC402/error_no_space.png)


Therefore, I've been exploring various ways to optimize the process and researching to find any possible solutions. 


<br>

### 1. Clear Temporary Files
- Clearing any unnecessary data or logs that may consume disk space is a necessary task. This includes temporary files, cache, or old application logs. However, the process resulted in FALSE, indicating that the task was not fully successful.







In [0]:
# Try to reduce temporary files to secure more data usages
# RESULT: FALSE 

# dbutils.fs.rm('/path/to/temp/files', recurse=True)

### 2. Check Cluster Logs
After checking my cluster logs, I have identified a few steps to optimize the clusters.

- Cluster Management: With more nodes, cluster management becomes more complex. Issues such as network partitions or synchronization become more prominent. But when I confirmed my clusters, the max workers number was 1, and I could not fix this.

<img src="/files/tables/DSCC402/GC_Allocation_Failure.png" alt="error_no_space" width="75%">
<img src="/files/tables/DSCC402/configuration.png" alt="error_no_space" width="75%">





### 3. Minor Optimizations
- Removed & Rewrite Checkpoints & Delta Table: Ensure the checkpoint directory is not accumulating excessive metadata or checkpoint files that consume disk space. I removed checkpoint data and cleaned old checkpoints since the streaming was not initiated. 

In [0]:
# REMOVE PREVIOUS DATA WHENEVER WE START
# RESUMT : TRUE

# dbutils.fs.rm(USER_DIR + "/bronze.delta", True)
# dbutils.fs.rm(USER_DIR + "/bronze.checkpoint", True)
# dbutils.fs.rm(USER_DIR + "/silver.delta", True)
# dbutils.fs.rm(USER_DIR + "/silver.checkpoint", True)
# dbutils.fs.rm(USER_DIR + "/gold.delta", True)
# dbutils.fs.rm(USER_DIR + "/gold.checkpoint", True)

### Conclusion

Due to the continuous problems I faced while creating the Gold Delta Table, I have realized that maintaining consistency in log files is important to prevent data spills. 
Here are some other ways that I could suggest: 
- Skewed Data: Symptoms of the issue will be an uneven distribution of tasks, where some tasks take much longer than others. To avoid this, examine the Stages tab for task durations and size variations, and look at the Input Size / Records and Shuffle Read/Write sizes. To optimize, we can implement custom partitioning or increase the number of partitions and use salting techniques for highly skewed keys.
- Serialization: This could happen when there are high task deserialization times and large serialized data sizes. We can confirm this at the task deserialization time in the Executors tab. Possible Optimizations would be minimizing the data that needs to be serialized across the network.